# **Дроп фрод основные инструменты**
- **ноутбуки лучше просматривать** на Github pages, т.к. при просмотре прямо в репозитории могут быть проблемы с отображением, например, обрезка вывода с широкими датафреймами. **Если в адресной строке браузера есть** `iaroslav-dzh.github.io`, то вы уже на Github pages.  
Ссылки:
    -  **[Ссылка](PASTE_HERE)** на этот ноутбук
    -  **[Ссылка](https://iaroslav-dzh.github.io/fraud_txns_simulator/)** на страницу со всеми ноутбуками
 
**Вступление к генерации дроп фрода**  
Это начало демонстрации генерации дроп фрода - далее будет еще 2 ноутбука о генерации дроп фрода, т.к. этот раздел получился более обширным.  
При написании генерации дроп фрода я полностью решил перейти на классы как основу кода, вместо функций.  

Дропов будет два типа: 
- распределители (distributors): поведение основано на регулярном получении денег и их распределении - перевод, снятие, покупка криптовалюты.
- покупатели (purchasers): поведение основано на регулярном получении денег и покупке на них товаров - отмыв денег.

**Информация о ноутбуке**  
- В этом ноутбуке базовые части генерации:
    - создание конфиг классов
    - управление счетами
    - генерация сумм транзакций
    - генерация времени транзакций
    - управление поведением дропа

In [4]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import os
# import re
# import geopandas as gpd
# from scipy.stats import truncnorm, norm
# from collections import defaultdict
# import math
import pyarrow
import yaml
from typing import Union

In [7]:
# Функции генерации паттернов времени и базовые функции времени созданные ранее
from data_generator.general_time import *
from data_generator.utils import load_configs

In [5]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [6]:
os.chdir("..")
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

In [8]:
# Базовые конфиги
base_cfg = load_configs("./config/base.yaml")
# Настройки легальных транзакций
legit_cfg = load_configs("./config/legit.yaml")
# Общие настройки фрода
fraud_cfg = load_configs("./config/fraud.yaml")
# Настройки для дроп фрода
drops_cfg = load_configs("./config/drops.yaml")
# Настройки времени
time_cfg = load_configs("./config/time.yaml") 

# Пути к файлам
data_paths = base_cfg["data_paths"]

<br><br>

# **Создание конфиг классов с конфигами и данными для генерации**
<br>

## 1. Класс `DropConfigBuilder`
- модуль `data_generator.fraud.drops.build.config`. **[Ссылка](https://github.com/iaroslav-dzh/fraud_txns_simulator/blob/main/data_generator/fraud/drops/build/config.py)** на файл в Github
- 
**Условия**  
- до этого были сгенерированы легальные и compromised client fraud транзакции

**Основная логика**
- создает объект `DropDistributorCfg` или `DropPurchaserCfg` для дропов распределителей или покупателей соотвественно
- суть такая же как у конфиг билдеров для легальных и compromised client fraud транзакций

In [9]:
# директорию текущего запуска генератора возьмем из предыдущего ноутбука. Т.к. нужны данные того, что сгенерировано до дроп фрода
run_dir = './data/generated/history/generation_run_2025-07-25_121029'

In [11]:
from data_generator.fraud.drops.build.config import DropConfigBuilder

# объект самого конфиг билдера
cfg_build = DropConfigBuilder(base_cfg=base_cfg, legit_cfg=legit_cfg, time_cfg=time_cfg, fraud_cfg=fraud_cfg, \
                              drop_cfg=drops_cfg, run_dir=run_dir)

# Конфиг класс для дропов распределителей
dist_configs = cfg_build.build_dist_cfg()
# Конфиг класс для дропов покупателей
purch_configs = cfg_build.build_purch_cfg()

### **Демонстрация**

In [12]:
# Клиенты семплированные под дропов распределителей т.е. они будут дропами
dist_configs.clients.head(2)

,client_id,birth_date,sex,region,city,lat,lon,city_id,home_ip
0,4344,1950-06-09,male,Свердловская,Екатеринбург,56.838633,60.605489,54,2.60.16.18
1,2692,1950-05-24,male,Ставропольский,Ставрополь,45.044544,41.969017,44,2.60.9.247


In [13]:
# Клиенты семплированные под дропов покупателей т.е. они будут дропами
purch_configs.clients.head(2)

,client_id,birth_date,sex,region,city,lat,lon,city_id,home_ip
0,901,1944-12-14,male,Смоленская,Смоленск,54.786717,31.815337,39,2.60.3.87
1,3839,1962-06-29,female,Ростовская,Таганрог,47.209491,38.935154,65,2.60.14.50


In [14]:
# таблица со счетами клиентов и информацией о том дропы ли они.
# клиент помечается как дроп только когда непосредственно идет генерация его активности
dist_configs.accounts.head(2)

,client_id,account_id,is_drop
0,1,10000,False
1,2,10001,False


<br><br>

# **Управление счетами транзакций. Генерация сумм транзакций и времени**
<br>

## 1. Класс `DropAccountHandler`
- модуль `data_generator.fraud.drops.base`. **[Ссылка](https://github.com/iaroslav-dzh/fraud_txns_simulator/blob/main/data_generator/fraud/drops/base.py)** на файл в Github 
- Управление счетами переводов, статусами дропов в таблице accounts

**Основной функционал**
- запись номера счета текущего дропа в атрибут `account`
- случайный выбор счета для исходящего перевода если это дроп распределитель
- пометка текущего клиента, как дропа в таблице `accounts`

### **Демонстрация**

In [16]:
from data_generator.fraud.drops.base import DropAccountHandler

acc_hand = DropAccountHandler(configs=dist_configs)
# DropAccountHandler нуждается в передаче ему client_id, чтобы знать чей счет искать если нужен счет самого дропа 
# и чей счет исключить если это счет под исходящюю транзакцию
acc_hand.client_id = 1

Найти и записать счет текущего клиента в атрибут account

In [19]:
acc_hand.get_account(own=True)
acc_hand.account

np.int64(10000)

Выбрать случайный счет под исходящий перевод

In [18]:
acc_hand.get_account()

np.int64(23260)

Пометить текущего клиента, как дропа

In [20]:
# смотрим до вызова метода. Клиент не помечен как дроп: is_drop = False
accounts = acc_hand.accounts
own_id = acc_hand.client_id
accounts.query("client_id == @own_id")

,client_id,account_id,is_drop
0,1,10000,False


In [21]:
# Отмечаем клиента как дропа
acc_hand.label_drop()

# Снова смотрим на него в accounts
accounts.query("client_id == @own_id")

,client_id,account_id,is_drop
0,1,10000,True


<br>

## 2. Класс `DropAmountHandler`
- модуль `data_generator.fraud.drops.base`. **[Ссылка](https://github.com/iaroslav-dzh/fraud_txns_simulator/blob/main/data_generator/fraud/drops/base.py)** на файл в Github

**Основной функционал**
- генерация сумм транзакций: вх./исх. переводов, снятий, покупок:
    - генерирует сумму в соответствии с конфигами в `drops.yaml` и в соответствии с переданными аргументами, например:
        - генерация сумм для переводов, снятий или покупок частями - в соответствии с лимитами в конфиге `drops.yaml`
        - генерация сумм для перевода всего баланса
        - уменьшение суммы следующего перевода если текущий перевод отклонен
- управление балансом текущего дропа:
    - начисление при успешной входящей транзакции
    - списание при успешной исх. транз-ции - перевод, снятие, покупка.
    - неизменение баланса если вх./исх. транз-ция отклонена 

### **Демонстрация**

In [22]:
from data_generator.fraud.drops.base import DropAmountHandler

amt_hand = DropAmountHandler(configs=dist_configs)

**Метод `receive`**
- генерация суммы входящего перевода
- изменение баланса на сумму перевода если транз. не отклонена

In [25]:
# текущий баланс
amt_hand.balance

0

In [27]:
# сумма вх. перевода. Успешная транз
amt_hand.receive(declined=False)

np.float64(30700.0)

In [28]:
# Баланс
amt_hand.balance

np.float64(30700.0)

In [29]:
# сумма вх. перевода. Отклоненная транз
amt_hand.receive(declined=True)

np.float64(34300.0)

In [30]:
# Баланс
amt_hand.balance

np.float64(30700.0)

**Метод `one_operation`**
- объединяет в себе несколько методов. Является конечным методом для генерации суммы транзакции и обновления баланса
- возвращает сумму транзакции

In [31]:
# Баланс
amt_hand.balance

np.float64(30700.0)

In [32]:
# Например текущая транзакция онлайн, она не отклонена и нужно перевести только часть баланса
amt_hand.one_operation(online=True, declined=False, in_chunks=True)

np.float64(20000.0)

In [33]:
amt_hand.balance

np.float64(10700.0)

**Пример с уменьшением суммы если транзакция отклонена**
- каждая сумма будет меньше предыдущей, но не меньше минимального значения, которые выставлено в конфигах `drops.yaml`

In [42]:
amt_hand.reset_cache(all=True)
amt_hand.balance = 47000
print(f"Баланс перед запуском цикла: {amt_hand.balance}")

all_ops = []
while True:
    one_op = int(amt_hand.one_operation(online=True, declined=True, in_chunks=True))
    
    all_ops.append(one_op)
    if len(all_ops) == 5:
        break
print(f"""Итоговый баланс: {amt_hand.balance}
Все созданные суммы: {all_ops}""")

Баланс перед запуском цикла: 47000
Итоговый баланс: 47000
Все созданные суммы: [24000, 18000, 12000, 6000, 3000]


<br>

## 3. Класс `DropTimeHandler`
- модуль `data_generator.fraud.drops.time`. **[Ссылка](https://github.com/iaroslav-dzh/fraud_txns_simulator/blob/main/data_generator/fraud/drops/time.py)** на файл в Github

**Основной функционал**
- управление временем транзакций дропа. Генерация времени  

**Основная логика**
1. время первой транзакции берется случайно из диапазона timestamp-ов
2. создает время транзакций в соответсвии с лимитами активности для дропов выставленных в `drops.yaml`
3. есть лимит на входящие транзакции и исходящие в периоде активности
4. период активности это когда дроп "работает" с деньгами - переводит, снимает, покупает.
5. период измеряется по кол-ву сделанных транзакций. Если достигнут какой-то из лимитов - входящие/исходящие - то дроп берет паузу в активности на период установленный в конфигах и +/- случайную дельту времени установленную также в конфигах. Пауза считается от первой транзакции в периоде, например новый период дроп перевел деньги в 12:00, и достиг лимита на исх. операции в текущем периоде. Берется время 12:00 и к нему прибавляется указанный лаг по времени, например 24 часа, и случайная положительная или отрицательная дельта, например -1 час. То есть дроп возобновляет активность в 12:00 + 24 часа + (-1) час = 11:00 следующего дня. Это будет транзакция уже нового периода активности.
6. если лимит не достигнут то время текущей транзакции также основывается на времени предыдущей: прибавляется случайная положительная дельта времени.

### **Демонстрация**

In [49]:
from data_generator.fraud.drops.time import DropTimeHandler

time_hand = DropTimeHandler(configs=dist_configs)

**Метод `get_txn_time`**
- основной метод класса. Генерирует время для вх. и исх. транз-ций
- внизу будет симулирована генерация под активность дропа. Так время ведет себя и при полноценной генерации активности дропа
- можно заметить что время между транзакциями `time_diff` периодами составляет не более 3-х часов и потом следует продолжительный перерыв в 15+ часов. Это и есть перерыв между периодами активности дропа. Условно, дроп "работает" какое-то время и уходит на долгий перерыв.

In [50]:
all_times = []

in_txns = 0
txn_time, _ = time_hand.get_txn_time(receive=True, in_txns=in_txns) # первая транзакция. Входящая
in_txns += 1
all_times.append(txn_time)

for _ in range(12): # цикл исходящих транзакций.
    txn_time, _ = time_hand.get_txn_time(receive=False, in_txns=in_txns)
    all_times.append(txn_time)
    
all_times = pd.DataFrame({"txn_time":all_times})
all_times["time_diff"] = all_times.txn_time - all_times.txn_time.shift(1)
all_times

,txn_time,time_diff
0,2025-01-14 09:27:00,NaT
1,2025-01-14 10:35:00,0 days 01:08:00
2,2025-01-14 12:17:00,0 days 01:42:00
3,2025-01-14 13:11:00,0 days 00:54:00
4,2025-01-14 16:10:00,0 days 02:59:00
5,2025-01-14 18:55:00,0 days 02:45:00
6,2025-01-15 10:22:00,0 days 15:27:00
7,2025-01-15 11:52:00,0 days 01:30:00
8,2025-01-15 13:15:00,0 days 01:23:00
9,2025-01-15 15:02:00,0 days 01:47:00


<br><br>

# **Управление поведением дропов**
<br>

## 1. Класс `DistBehaviorHandler`  
- модуль `data_generator.fraud.drops.behavior`. **[Ссылка](https://github.com/iaroslav-dzh/fraud_txns_simulator/blob/main/data_generator/fraud/drops/behavior.py)** на файл в Github

**Условия**
- подразумевается, что дроп получает сумму и распределяет её до получения новой

**Основной функционал**
- управление поведением дропов распределителей
- случайный выбор сценария распределения полученных денег - перевод всего, снятие всего, перевод частями, снятие+перевод частями - выбор сценария имеет зависимость от баланса. Например если баланс больше установленной суммы, то не может быть выбран сценарий с одной операцией.
- случайно определять будет ли транзакция покупкой криптовалюты
- определение кол-ва попыток совершить операцию после первой отклоненной транзакции
- вычитание попыток когда дроп пытается совершать операции после первой отклоненной транз.
- идентификация того нужно ли пытаться еще после последней отклоненной транзакции (это может быть первая отклоненная транз. если выпало 0 попыток)

### **Демонстрация**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [3]:
from data_generator.indev import DropConfigBuilder
# from data_generator.configs import DropDistributorCfg
from data_generator.fraud.drops.behavior import DistBehaviorHandler
from data_generator.fraud.drops.base import DropAmountHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
config3 = drop_cfg_build.build_dist_cfg()
drop_amt1 = DropAmountHandler(configs=config3)
dist_behav1 = DistBehaviorHandler(configs=config3, amt_hand=drop_amt1)

**`sample_scenario`**

`split_rate` = 0.7  
Тест `large_balance`: баланс больше чем `DistBehaviorHandler.trf_max`

In [1242]:
# Вероятности сценариев для large_balance
split_rate = 0.7
p_atm_and_trf = split_rate * 0.5
p_split_trf = split_rate * 0.5
p_atm = 1 - split_rate
print(f"""atm+transfer prob: {p_atm_and_trf}
split_transfer prob: {p_split_trf}
atm prob: {p_atm:.3f}""")
assert sum([p_atm_and_trf, p_split_trf, p_atm]) == 1

atm+transfer prob: 0.35
split_transfer prob: 0.35
atm prob: 0.300


In [4]:
all_scens1 = []
i = 0
while i < 2000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_max + 1000
    assert drop_amt1.balance > dist_behav1.trf_max, "Balance is below trf_max"
    dist_behav1.sample_scenario()
    # print(dist_behav1.scen)
    all_scens1.append(dist_behav1.scen)
    i += 1
all_scens_ser1 = pd.Series(all_scens1)

In [5]:
all_scens_ser1.value_counts(normalize=True)

split_transfer    0.3775
atm+transfer      0.3270
atm               0.2955
Name: proportion, dtype: float64

`split_rate` = 0.7  
Тест `atm_eligible`: баланс больше чем `DistBehaviorHandler.atm_min`, но меньше чем `DistBehaviorHandler.trf_max`

In [1245]:
# Вероятности сценариев для atm_eligible
split_rate = 0.7
p_atm_and_trf = split_rate * 0.5
p_split_trf = split_rate * 0.5
p_trf = (1 - split_rate) * 0.5
p_atm = (1 - split_rate) * 0.5
print(f"""atm+transfer prob: {p_atm_and_trf}
split_transfer prob: {p_split_trf}
transfer prob: {round(p_trf, 3)}
atm prob: {round(p_atm, 3)}""")
assert sum([p_atm_and_trf, p_split_trf, p_trf, p_atm]) == 1

atm+transfer prob: 0.35
split_transfer prob: 0.35
transfer prob: 0.15
atm prob: 0.15


In [6]:
all_scens2 = []
i = 0
while i < 2000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.atm_min + 1000
    assert drop_amt1.balance < dist_behav1.trf_max, "Balance exceeds trf_max"
    dist_behav1.sample_scenario()
    all_scens2.append(dist_behav1.scen)
    i += 1
all_scens_ser2 = pd.Series(all_scens2)

In [7]:
all_scens_ser2.value_counts(normalize=True)

atm+transfer      0.3455
split_transfer    0.3420
atm               0.1610
transfer          0.1515
Name: proportion, dtype: float64

`split_rate` = 0.7  
Тест `split_eligible`: баланс больше чем `DistBehaviorHandler.trf_min * 2`,  
но меньше чем `DistBehaviorHandler.atm_min` и `DistBehaviorHandler.trf_max`

In [8]:
# Вероятности сценариев для split_eligible
split_rate = 0.7
p_split_trf = split_rate
p_trf = 1 - split_rate
print(f"""split_transfer prob: {p_split_trf}
transfer prob: {round(p_trf, 3)}""")
assert sum([p_split_trf, p_trf]) == 1

split_transfer prob: 0.7
transfer prob: 0.3


In [9]:
all_scens3 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_min * 2
    assert drop_amt1.balance < dist_behav1.atm_min, "Balance exceeds atm_min"
    assert drop_amt1.balance < dist_behav1.trf_max, "Balance exceeds trf_max"
    dist_behav1.sample_scenario()
    all_scens3.append(dist_behav1.scen)
    i += 1
all_scens_ser3 = pd.Series(all_scens3)

In [10]:
all_scens_ser3.value_counts(normalize=True)

split_transfer    0.692333
transfer          0.307667
Name: proportion, dtype: float64

`split_rate` = 0.7  
Остальные случаи. баланс меньше чем `DistBehaviorHandler.trf_min * 2`

In [1251]:
all_scens4 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_min + 1000
    assert drop_amt1.balance < dist_behav1.trf_min * 2, "Balance exceeds the limit"
    dist_behav1.sample_scenario()
    all_scens4.append(dist_behav1.scen)
    i += 1
all_scens_ser4 = pd.Series(all_scens4)

In [1252]:
all_scens_ser4.value_counts(normalize=True)

transfer    1.0
Name: proportion, dtype: float64

**`in_chunks_val`**

In [1253]:
# Список сценариев
# "transfer"
# "atm"
# "atm+transfer"
# "split_transfer"

dist_behav1.scen = "split_transfer"
dist_behav1.in_chunks_val()
dist_behav1.in_chunks

True

**`guide_scenario`**

In [1256]:
# Список сценариев
# "atm+transfer"
# "atm"
# "split_transfer"
# "transfer"

dist_behav1.scen = "transfer"
dist_behav1.guide_scenario()
dist_behav1.online

True

**`to_drop`**

In [1257]:
print(dist_behav1.to_drop_rate)
to_drop_list1 = []
for _ in range(100):
    to_drop_list1.append(dist_behav1.to_drop)
to_drop_ser1 = pd.Series(to_drop_list1)
to_drop_ser1.value_counts(normalize=True)

0.1


False    0.92
True     0.08
Name: proportion, dtype: float64

**`to_crypto`**

In [1258]:
dist_behav1.online = True
print(dist_behav1.crypto_rate)
to_drop_list2 = []

for _ in range(100):
    to_drop_list2.append(dist_behav1.to_drop)
to_drop_ser2 = pd.Series(to_drop_list2)
to_drop_ser2.value_counts(normalize=True)

0.1


False    0.91
True     0.09
Name: proportion, dtype: float64

**`stop_after_decline`**

In [424]:
dist_behav1.attempts = 2
dist_behav1.stop_after_decline(declined=True)

False

**`reset_cache`**

In [1261]:
dist_behav1.attempts = 3
dist_behav1.scen = "atm"
dist_behav1.online = True
dist_behav1.in_chunks = False
dist_behav1.reset_cache(all=True)
dist_behav1.attempts, dist_behav1.scen, dist_behav1.online, dist_behav1.in_chunks

(0, None, None, None)

**`attempts_after_decline`**  
self.online = True, self.online = False

In [1264]:
att_vals1 = []
dist_behav1.online = True

for _ in range(100):
    dist_behav1.attempts_after_decline(declined=True)
    att_vals1.append(dist_behav1.attempts)
att_vals_ser1 = pd.Series(att_vals1)

In [1265]:
att_vals_ser1.agg(["min","mean", "max"])

min     0.00
mean    1.94
max     4.00
dtype: float64

**`deduct_attempts`**

In [434]:
dist_behav1.attempts = 1
dist_behav1.deduct_attempts(declined=True, receive=True)
dist_behav1.attempts

1

**Расчет вероятностей сценариев в `DistBehaviorHandler`**

In [ ]:
# inbound_amt: 
#   low: 5000
#   high: 100000
#   mean: 30000
#   std: 20000

In [308]:
# Баланс генерируется из обрезанного нормального распределения
in_low1 = 5000
in_high1 = 100000
mean1 = 30000
std1 = 20000
a1 = (in_low1 - mean1) / std1
b1 = (in_high1 - mean1) / std1

split_rate = 0.7
# Есть лимиты для сумм операций
atm_min1 = 10000 # минимальная сумма для снятия. Частями или целиком определяется split_rate
trf_lim = 40000 # максимальная сумма для перевода. Если больше - дробим на части
trf_min = 3000*2 # минимальная сумма для перевода по частям. Если меньше - переводим целиком

In [309]:
# вероятность что сумма будет больше trf_lim
p_trf_lim = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_lim), 3)
p_trf_lim

np.float64(0.345)

In [310]:
# Вероятность что сумма больше atm_min1 но меньше trf_lim
p_atm_min = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=atm_min1) - p_trf_lim, 3)
p_atm_min

np.float64(0.596)

In [311]:
# вероятность что сумма больше trf_min, но меньше trf_lim и atm_min1
p_trf_min = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_min) - p_trf_lim - p_atm_min, 3)
p_trf_min

np.float64(0.048)

In [312]:
split_rate = 0.7 # доля когда будем переводить частями

(p_trf_lim + p_atm_min + p_trf_min) * split_rate

np.float64(0.6922999999999999)

In [313]:
round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_min), 3) * split_rate

np.float64(0.6922999999999999)

In [315]:

# conditions = [
#             (balance > self.trf_lim, ["atm","split_transfer", "atm+transfer"]),
#             (balance >= self.atm_min, ["transfer", "atm", "split_transfer", "atm+transfer"]),
#             (balance >= self.trf_min * 2, ["transfer","split_transfer"]),
#             "transfer",
#         ]
# Вероятность случаев перевода частями если по условиям выше (синтаксически неправильно) и без split_rate
p_trf_lim * 2/3 + p_atm_min * 1/2 + p_trf_min * 1/2

np.float64(0.552)

In [645]:

# conditions = [
#             (balance > self.trf_lim, ["atm","split_transfer", "atm+transfer"]),
#             (balance >= self.atm_min, ["transfer", "atm", "split_transfer", "atm+transfer"]),
#             (balance >= self.trf_min * 2, ["transfer","split_transfer"]),
#             "transfer",
#         ]
# Вероятность случаев перевода в крипту по условиям выше (синтаксически неправильно)
crypto_rate = 0.14
(p_trf_lim * 2/3 + p_atm_min * 3/4 + p_trf_min) * crypto_rate

np.float64(0.1015)

## Класс `PurchBehaviorHandler`  
- модуль `data_generator.fraud.drops.behavior`
- Управление поведением дропа покупателя

In [ ]:
class PurchBehaviorHandler:
    """
    Управление поведением дропа покупателя. Выбор сценария.
    Контроль попыток после откл. транз.
    ----------
    Атрибуты:
    --------
    scen: str. Выбранный сценарий поведения. По умолчанию None.
    amt_hand: DropAmountHandler. Управление балансом и суммами переводов.
    amt_min: int. Мин. сумма покупки.
    amt_max: int. Мак. сумма покупки.
    split_rate: float. Доля случаев когда полученная сумма будет распределена по частям.
                При условии что полученная сумма пройдет по лимитам.
    online: bool. Какая должна быть транзакция: онлайн или оффлайн.
            По умолчанию True. Не подразумевается оффлайна на данный момент.
    in_chunks: bool. Использует ли дроп полученные деньги по частям. По умолчанию None.
    attempts: int. Сколько попыток совершить операцию будет сделано 
               дропом после первой отклоненной транзакции. По умолчанию 0.
    attempts_cfg: dict. Лимиты возможных попыток
    """

    def __init__(self, configs: DropPurchaserCfg, amt_hand: DropAmountHandler):
        """
        configs: DropPurchaserCfg. Конфиги и данные для создания дроп транзакций.
        amt_hand: DropAmountHandler. Отсюда узнаем текущий баланс.
        
        """
        self.scen = None
        self.amt_hand = amt_hand
        self.amt_min = amt_hand.chunks["rcvd_small"]["min"]
        self.amt_max = configs.amt_max
        self.split_rate = configs.split_rate
        self.online = True
        self.in_chunks = None
        self.attempts = 0
        self.attempts_cfg = configs.attempts


    def sample_scenario(self):
        """
        Выбор сценария поведения дропа с учётом текущего баланса 
        и актуальных лимитов.
        """
        balance = self.amt_hand.balance
        split = np.random.uniform(0,1) <= self.split_rate

        # Минимальный баланс для переводов по частям будет самый маленький возможный размер чанка
        # self.amt_min умноженный на 2
        large_balance = balance > self.amt_max
        split_eligible = balance >= self.amt_min * 2

        # Список: (условие, возможный сценарий)
        conditions = [
            (large_balance, "split_money"),
            (split_eligible and split, "split_money")
        ]

        for cond, scen in conditions:
            if cond:
                self.scen = scen
                return
        # Если ни одно из условий не сработало
        self.scen = "one_purchase"


    def in_chunks_val(self):
        """
        Запись значения атрибута in_chunks в зависимости от
        сценария.
        """
        scen = self.scen

        if scen == "split_money":
            self.in_chunks = True
            return
        if scen == "one_purchase":
            self.in_chunks = False


    def stop_after_decline(self):
        """
        Будет ли дроп пытаться еще после отклоненной операции
        или остановится.
        ---------------
        """
        declined_txns = self.amt_hand.declined_txns

        if declined_txns == 0:
            return False
        if self.attempts == 0:
            return True
        if self.attempts > 0:
            return False


    def attempts_after_decline(self):
        """
        Рандомизация количества попыток дропа совершить операцию после первой
        отклоненной транзакции.
        Количество попыток определяется когда есть только одна отклоненная транзакция
        на данный момент. Т.к. нужно задать число попыток только один раз.
        """
        declined_txns = self.amt_hand.declined_txns
        
        if declined_txns != 1:
            return
        
        att_min = self.attempts_cfg["min"]
        att_max = self.attempts_cfg["max"]
        self.attempts = np.random.randint(att_min, att_max + 1)
        # print("attempts", self.attempts)

    
    def deduct_attempts(self):
        """
        Вычитание попытки исходящей транзакции совершенной
        после первой отклоненной транзакции.
        """
        declined_txns = self.amt_hand.declined_txns

        if declined_txns <= 1:
            return
        if self.attempts == 0:
            return
        
        # Иначе вычитаем попытку
        self.attempts -= 1


    def reset_cache(self, all=False):
        """
        Сброс кэшированных данных
        -------------
        all: bool. Если False, не будут сброшены self.attempts
        """
        self.scen = None
        self.online = True
        self.in_chunks = None
        if not all:
            return
        self.attempts = 0

**Тест `PurchBehaviorHandler`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [2]:
from data_generator.indev import DropConfigBuilder
from data_generator.fraud.drops.behavior import PurchBehaviorHandler
from data_generator.fraud.drops.base import DropAmountHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
configs = drop_cfg_build.build_purch_cfg()
drop_amt1 = DropAmountHandler(configs=configs)
purch_behav1 = PurchBehaviorHandler(configs=configs, amt_hand=drop_amt1)

`split_rate` = 0.7  
Тест `large_balance`: баланс больше чем amt_max

In [3]:
all_scens1 = []
i = 0
while i < 2000:
    drop_amt1.reset_cache()
    drop_amt1.balance = purch_behav1.amt_max + 1000
    assert drop_amt1.balance > purch_behav1.amt_max, "Balance is below amt_max"
    purch_behav1.sample_scenario()
    # print(purch_behav1.scen)
    all_scens1.append(purch_behav1.scen)
    i += 1
all_scens_ser1 = pd.Series(all_scens1)

all_scens_ser1.value_counts(normalize=True)

split_money    1.0
Name: proportion, dtype: float64

`split_rate` = 0.7  
Тест `split_eligible`: баланс больше чем `self.amt_min * 2`,  
но меньше чем `self.amt_max`

In [5]:
# Вероятности сценариев для split_eligible
split_rate = 0.7
p_split = split_rate
p_one = 1 - split_rate
print(f"""split_money prob: {p_split}
one_purchase prob: {round(p_one, 3)}""")
assert sum([p_split, p_one]) == 1

split_money prob: 0.7
one_purchase prob: 0.3


In [6]:
all_scens3 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = purch_behav1.amt_min * 2
    assert drop_amt1.balance < purch_behav1.amt_max, "Balance exceeds amt_max"
    purch_behav1.sample_scenario()
    all_scens3.append(purch_behav1.scen)
    i += 1
all_scens_ser3 = pd.Series(all_scens3)

all_scens_ser3.value_counts(normalize=True)

split_money     0.697333
one_purchase    0.302667
Name: proportion, dtype: float64

`split_rate` = 0.7  
баланс меньше чем `amt_min * 2`

In [8]:
all_scens4 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = purch_behav1.amt_min + 1000
    assert drop_amt1.balance < purch_behav1.amt_min * 2, "Balance exceeds the limit"
    purch_behav1.sample_scenario()
    all_scens4.append(purch_behav1.scen)
    i += 1
all_scens_ser4 = pd.Series(all_scens4)

all_scens_ser4.value_counts(normalize=True)

one_purchase    1.0
Name: proportion, dtype: float64

# **`in_chunks_val`**

In [4]:
# Список сценариев
# "split_money"
# "one_purchase"

purch_behav1.scen = "one_purchase"
purch_behav1.in_chunks_val()
purch_behav1.in_chunks

False

**`stop_after_decline`**

In [8]:
purch_behav1.attempts = 0
purch_behav1.stop_after_decline(declined=True)

True

**`reset_cache`**

In [9]:
purch_behav1.attempts = 3
purch_behav1.scen = "split_money"
purch_behav1.online = True
purch_behav1.in_chunks = False
purch_behav1.reset_cache(all=True)
purch_behav1.attempts, purch_behav1.scen, purch_behav1.online, purch_behav1.in_chunks

(0, None, True, None)

**`attempts_after_decline`**  
declined = True/False

In [3]:
att_vals1 = []

for _ in range(100):
    purch_behav1.attempts_after_decline(declined=True)
    att_vals1.append(purch_behav1.attempts)
att_vals_ser1 = pd.Series(att_vals1)

att_vals_ser1.agg(["min","mean", "max"])

min     0.00
mean    1.87
max     4.00
dtype: float64

**`deduct_attempts`**

In [6]:
purch_behav1.attempts = 4
purch_behav1.deduct_attempts(declined=True, receive=False)
purch_behav1.attempts

3

## Конец информативной части

# Старый код, функции и т.д.